In [33]:
from python_files.open_fits_file import openfits
path_to_data = 'C:/Users/andre/MasterCode/Grizli_transient_data/obs17'
names, Data, Header= openfits(path_to_data)

In [34]:
#        old      new      diff     150      200      277     356       444
data = [Data[2], Data[1], Data[0], Data[3], Data[4], Data[5], Data[6], Data[7]]
header = [Header[2], Header[1], Header[0], Header[3], Header[4], Header[5], Header[6], Header[7]]

In [35]:
import numpy as np
import matplotlib.pyplot as plt
def plot_image(Data, ep=1):
    file = Data
    mean = np.mean(file)
    std = np.std(file)
    plt.imshow(file, vmin=mean-ep*std, vmax=mean+ep*std)

In [36]:
#Confirmation that 0 = old, 2 = diff
# plt.figure(figsize=(16,8))
# plt.subplot(1, 3, 1)
# plot_image(data[0][5000:10000,5000:10000])
# plt.subplot(1, 3, 2)
# plot_image(data[1][5000:10000,5000:10000])
# plt.subplot(1, 3, 3)
# plot_image(data[2][5000:10000,5000:10000])


In [37]:
import csv

# Open the CSV file
path_to_CNN_predictions = 'C:/Users/andre/MasterCode/Code/Predicted_transients/Predicted_transients_'+path_to_data[-5:]+'.csv'
with open(path_to_CNN_predictions, mode='r') as file:
    transients = []
    non_transients = []
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    # Iterate through each row in the CSV file
    for row in csv_reader:
        if row[3] == '1.0':
            transients.append(row)
        if row[3] == '0.0':
            non_transients.append(row)
            #print(row)  # Each row is a list of strings

In [38]:
from python_files.Find_magnitude import magnitude_in_each_band, flux_from_mag_AB, magnitude_in_each_band_with_aperture
from python_files.Find_Skycoords import find_skycoords
from python_files.convert_sky_phy import find_skycoord_in_physical

In [39]:
transients_skycoords = find_skycoords(transients)

#The positions are found in the difference images, so we need to use the header of the difference image to convert them to physical coordinates.
xy_transients = find_skycoord_in_physical(header[2], transients_skycoords)

mag_transients = magnitude_in_each_band_with_aperture(transients_skycoords, data, header)



In [40]:
def round_mag(magnitudes, r=3):
    rounded_magnitude = []
    for mag in magnitudes:
        rounded_mag = []
        for m in mag:
            if m == 99 or m == -99:
                rounded_m = round(m,r)
            else:
                rounded_m = round(m.value,r)
            rounded_mag.append(rounded_m)
        rounded_magnitude.append(rounded_mag)
    return rounded_magnitude
    
def add_row_to_table(ID, VIflag, skycoords, phycoords, magnitudes, table):
    for i in range(len(skycoords)):
        mag = magnitudes[i]
        sky = skycoords[i]
        phy = phycoords[i]
        table.add_row([ID, sky.ra.value, sky.dec.value, phy[0],phy[1], f'{VIflag}', mag[0], mag[1], mag[2], mag[3], mag[4], mag[5], mag[6], mag[7]])

In [41]:
rounded_mag = round_mag(mag_transients)

In [42]:
transients_with_magnitude = []
for i in range(len(transients_skycoords)):
    mag = rounded_mag[i]
    sky = transients_skycoords[i]
    phy = xy_transients[i]
    transients_with_magnitude.append([path_to_data[-5:], sky.ra.deg, sky.dec.deg, phy[0], phy[1], f'{1}', mag[0], mag[1], mag[2], mag[3], mag[4], mag[5], mag[6], mag[7]])

In [43]:
from python_files.Find_magnitude import find_limiting_magnitude

In [44]:
bkgmag_115 = find_limiting_magnitude('F115W', data[1], header[1], if_hist = 0)#, n_apertures=2000)
print(f'Limiting magnitude in F115W: 5 sigma: {bkgmag_115[2]}')

bkgmag_200 = find_limiting_magnitude('F200W', data[4], header[4], if_hist = 0)#, n_apertures=2000)
print(f'Limiting magnitude in F200W: 3 sigma: {bkgmag_200[1]}')

bkgmag_356 = find_limiting_magnitude('F356W', data[6], header[6], if_hist = 0)#, n_apertures=2000)
print(f'Limiting magnitude in F356W: 3 sigma: {bkgmag_356[1]}')

limiting_magnitudes = [bkgmag_115[2], bkgmag_200[1], bkgmag_356[1]]
#                         5 sigma       3 sigma        3 sigma


Limiting magnitude in F115W: 5 sigma: 26.75353483717402
Limiting magnitude in F200W: 3 sigma: 25.84375354475108
Limiting magnitude in F356W: 3 sigma: 28.458150752251967


In [45]:
def verify_transient_in_two_filters(transient_csv, limiting_magnitude):
    """
    Keep only transients that are brighter than the limiting magnitude in F115W
    *and* in at least one of F200W or F356W.  A transient is never appended more
    than once.
    """
    real_transients = []
    lim = limiting_magnitude          # (F115W, F200W, F356W) limiting mags

    for tr in transient_csv:
        f115w_mag = float(tr[7])
        f200w_mag = float(tr[10])
        f356w_mag = float(tr[12])

        # First branch: F115W + F200W
        if f200w_mag != -99 and f115w_mag < lim[0] and f200w_mag < lim[1]:
            real_transients.append(tr)

        # Second branch: F115W + F356W, but only if not already added
        if (
            f356w_mag != -99
            and f115w_mag < lim[0]
            and f356w_mag < lim[2]
            and tr not in real_transients        # <- duplication guard
        ):
            real_transients.append(tr)

    return real_transients

In [46]:
verified_transients = verify_transient_in_two_filters(transients_with_magnitude, limiting_magnitudes)
print(len(verified_transients))

8


In [47]:
from python_files.convert_sky_phy import find_skycoord_in_physical_from_nonSkycoords
verified_skycoords = find_skycoord_in_physical_from_nonSkycoords(header[2], verified_transients)

In [48]:
from python_files.remove_dipoles_new import remove_dipoles_with_flux, plot_non_dipoles, plot_dipoles

removed_dipoles, dipoles = remove_dipoles_with_flux(verified_skycoords, data[2])
print(len(removed_dipoles))

5


In [50]:
#Check that dipoles are removed:
#plot_non_dipoles(removed_dipoles=removed_dipoles , data=data, header=header)
#plot_dipoles(removed_dipoles=dipoles , data=data, header=header)

In [51]:
from astropy.wcs import WCS
def convert_to_skycoord(header, coords):
    sky_coords = []
    for coord in coords:
        x, y = coord
        w = WCS(header)
        ra_dec = w.pixel_to_world(x, y)
        sky_coords.append(ra_dec)
    return sky_coords

skycoords_no_dipoles = convert_to_skycoord(header[2], removed_dipoles)

In [52]:
from python_files.Plot_all_filters import find_diff_bands_cutout_plot

In [53]:
#find_diff_bands_cutout_plot(path_to_data[-5:], data, header, skycoords_no_dipoles)

In [54]:
mag_no_dipoles = magnitude_in_each_band_with_aperture(skycoords_no_dipoles, data, header)

In [55]:
from astropy.io import fits

with fits.open('C:/Users/andre/MasterCode/CosmosWebCatalogue/COSMOSWeb_master_v3.1.0-sersic-cgs_err-calib_LePhare.fits') as hdul:

    table = hdul[1].data
    header0 = hdul[0].header 
    header1 = hdul[1].header

rows = []
cosmos_RA = []
cosmos_DEC = []

for row in table:          # iterate directly over the items in `table`
    rows.append(row)       # save the row
    cosmos_RA.append(row['RA_DETEC'])
    cosmos_DEC.append(row['DEC_DETEC'])

In [56]:
float_skycoords_no_dipoles= [(float(x.ra.deg),float(x.dec.deg)) for x in skycoords_no_dipoles] 

In [57]:
from python_files.Find_closest_object import find_closest_objects_kdtree

obj, dist, z_final, z_PDF, z_u68, z_l68 = find_closest_objects_kdtree(rows, cosmos_RA, cosmos_DEC, float_skycoords_no_dipoles)


In [58]:
import csv
import os

# Create a folder to store csv file
output_folder_csvfile = 'C:/Users/andre/MasterCode/Code/cross_matched_transients'
os.makedirs(output_folder_csvfile, exist_ok=True)  # Creates the folder if it doesn't exist

csv_file_name = os.path.join(output_folder_csvfile, path_to_data[-5:]+'.csv')

with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'RA', 'DEC', 'x', 'y', 'CNN_prediction', 'F115W_old', 'F115W_new', 'F115W_diff', 'F150W', 'F200W', 'F277W', 'F356W', 'F444W', 'COSMOS-Web Catalog ID', 'Distance to host (in arcsec)', 'z_Final', 'z_PDF', 'z_u68', 'zl68'])

    for i in range(len(skycoords_no_dipoles)):
        sky = skycoords_no_dipoles[i]
        x, y = removed_dipoles[i]
        mag = mag_no_dipoles[i]
        ob = obj[i]
        dis = dist[i]
        z_fin = z_final[i]
        z = z_PDF[i]
        z_u = z_u68[i]
        z_l = z_l68[i]
        writer.writerow([
            f'{path_to_data[-5:]}_{i}', sky.ra.deg, sky.dec.deg, f'{x:.4f}', f'{y:.4f}', 1,
            f'{mag[0]:.4f}', f'{mag[1]:.4f}', f'{mag[2]:.4f}', f'{mag[3]:.4f}',
            f'{mag[4]:.4f}', f'{mag[5]:.4f}', f'{mag[6]:.4f}', f'{mag[7]:.4f}',
            f'{ob}', f'{dis}', f'{z_fin}', f'{z}', f'{z_u}', f'{z_l}' 
        ])